#**LAB №1 Data Collection**
**Максимов Антон 17ПМИ**

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
cd /content/gdrive/'My Drive'/DataScience/NLP

/content/gdrive/My Drive/DataScience/NLP


In [ ]:
!pip3 install --upgrade newspaper3k progressbar2
import newspaper
import json
import requests
import pandas as pd
from newspaper import Config
from newspaper import Article
from newspaper import Source
from newspaper.utils import BeautifulSoup
from time import sleep
from random import randint
from datetime import datetime
from multiprocessing.pool import ThreadPool
import progressbar


In [17]:
class NewsParser:
  
   def __init__(self, category, dates = None, filename=None):

     self.USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
     self.pool = ThreadPool(8)
     self.config = Config()
     self.config.browser_user_agent = self.USER_AGENT
     self.config.request_timeout = 10
     self.rubrics = ['russia', 'world', 'ussr', 'economics', 'forces', 'science', 'sport', 'culture', 'media']
     if dates is None:
       self.dates = pd.date_range(start="2018-01-01",end="2021-01-10")
     else:
       self.dates = dates
     if category in self.rubrics:
       self.category = category
     else:
       self.category = 'russia'
     print('Chosen category:',self.category)
     if filename is None:
       self.filename = 'data.json'
     else:
       self.filename = filename
     self.articles = []
     self.errors = []
     self.extraction_results = {}
     self.extraction_results['catalog'] = []

   def get_article(self,new):
    art_dict = {}
    url = 'https://lenta.ru'+new.find('div', {'class':'titles'}).a.attrs['href']
    try:
      article = Article(url, config=self.config)
      article.download()
      article.parse()
      response = requests.get(url)
      soup = BeautifulSoup(response.text, 'lxml')
      paragraphs = soup.find('div', attrs={"itemprop": "articleBody"})
      paragraphs = paragraphs.find_all('p')
      text = ''.join([p.get_text() for p in paragraphs])
      #title = soup.find('h1', 'b-topic__title').get_text()
      art_dict['article_id'] = url
      #art_dict['publish_date'] = article.publish_date.strftime("%Y/%m/%d")  #Можно добавить другие атрибуты (дату, описание), но в задании их нет(
      art_dict['title'] = article.title
      art_dict['category'] = self.category
      if not article.tags:
        try:
            tag = soup.find('a', 'item dark active')
            tag = re.match(r'\/rubrics\/[A-z0-9]+\/(([A-z0-9]+)\/)?', tag['href']).group(2)
            art_dict['tags'] = tag
        except Exception:
            tag = None
            art_dict['tags'] = None
      else:
        art_dict['tags'] = list(article.tags)
      art_dict['text'] = text
      #article_description = "".join({value for (key, value) in article.meta_data.items() if key == 'description'})
      #art_dict['description'] = article_description
      return art_dict
    except:
      self.errors.append(url)
      return {}
   
   def get_news(self,date):
    source = 'https://lenta.ru/rubrics/{}/{}/'.format(self.category, date.strftime("%Y/%m/%d"))
    response = requests.get(source)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    all_news_that_day = soup.find_all('div',{'class':'item news b-tabloid__topic_news'})
    for new in all_news_that_day:
      self.articles.append(self.get_article(new))
   
   def save_json(self):
     for art in self.articles:
        self.extraction_results['catalog'].append(art)
     with open(self.filename, 'w') as json_file:
        json.dump(self.extraction_results, json_file)
     print("Done!")

   def start(self):
     self.pool.map(self.get_news, self.dates)
     self.pool.close()
     self.pool.join()

     self.save_json()

**Пример работы**

In [18]:
cat = [ 'forces', 'science', 'sport', 'culture', 'media','economics']
for c in cat:
  parser = NewsParser(c, dates=pd.date_range(start="2020-06-01",end="2021-01-10"), filename = c+'.json') 
  parser.start()
  print("Category:",c)
  print("Length of articles:", len(parser.articles))

Chosen category: forces
Done!
Category: forces
Length of articles: 3184
Chosen category: science
Done!
Category: science
Length of articles: 3017
Chosen category: sport
Done!
Category: sport
Length of articles: 4427
Chosen category: culture
Done!
Category: culture
Length of articles: 2718
Chosen category: media
Done!
Category: media
Length of articles: 2849
Chosen category: economics
Done!
Category: economics
Length of articles: 4966


**По времени сбор занимает около двух часов (2,5), конкретно за этот промежуток**

In [32]:
with open('sport.json') as json_file:
    data = json_file.read()
    data = json.loads(data)

In [33]:
data["catalog"][0:5]

[{'article_id': 'https://lenta.ru/news/2020/07/12/britainpolice/',
  'category': 'sport',
  'tags': ['Уилфриду Заа'],
  'text': 'Британские полицейские арестовали 12-летнего мальчика, отправлявшего расистские оскорбления футболисту «Кристал Пэлас» Уилфриду Заа. Об этом сообщает ESPN.Проживающий в пригороде Бирмингема ребенок писал ивуарийцу в социальных сетях накануне матча лондонцев в рамках 35-го тура Английской премьер-лиги (АПЛ) против местной «Астон Виллы». Бирмингемский клуб уже назвал действия болельщика недопустимыми и пообещал вынести в отношении задержанного пожизненный запрет на посещение стадиона.Матч между «Кристал Пэлас» и «Астон Виллой» прошел в воскресенье, 12 июля. Встреча завершилась победой бирмингемцев со счетом 2:0.27-летний Заа в нынешнем сезоне провел 35 матчей в АПЛ. Игрок забил четыре мяча и сделал три голевые передачи.',
  'title': 'Британская полиция арестовала 12-летнего мальчика за оскорбления игрока АПЛ'},
 {'article_id': 'https://lenta.ru/news/2020/06/08/